<a href="https://colab.research.google.com/github/ViniciusBerti/20241_maua_ecm252_intro_git/blob/main/imt_ex4b_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Exercício Avaliativo 2 - 4o Bimestre**

In [ ]:
#@title **Identificação do Grupo**

#@markdown Nomes completos em ordem alfabética (*\<nome\>, \<RA\>*)
Aluno1 = 'ERICK EIJI NAGAO, 21.00690-3' #@param {type:"string"}
Aluno2 = 'IGOR IMPROTA MARTINEZ DA SILVA, 21.00834-5' #@param {type:"string"}
Aluno3 = 'GABRIEL HENRIQUE BACA RADO, 21.01286-5' #@param {type:"string"}
Aluno4 = 'RYUSKE HIDEAKI SATO, 21.00745-4' #@param {type:"string"}
Aluno5 = 'VINICIUS DE OLIVEIRA BERTI, 21.01219-9' #@param {type:"string"}
Aluno6 = 'None' #@param {type:"string"}




# **Tokenização e Clusterização**

## Introdução
A clusterização de textos tem uma série de usos, como o agrupamento de mensagens, textos ou notícias com características comuns. Os algoritmos tradicionais de clusterização, entretanto, requerem *features* numéricas e, portanto, a *tokenização* dos documentos.

## Este trabalho

Neste trabalho você vai analisar fazer a clusterização de mensagens de grupos de notícias e comparar a aderência (*acuracidade de clusterização*) dos agrupamentos obtidos com as classes previamente conhecidas dos grupos.

1. **A1 (8.0)** **Tokenização TF-IDF e Clusterização**. Empregue uma tokenização TF-IDF de sua escolha e faça ao menos dois modelos de clusterização (por exemplo k-médias e clusterização hierárquica). Tenha atenção e justifique os parâmetros da Tokenização (n-gramas por exemplo) e da clusterização (métrica de distância!). Analise e compare os resultados com as classes já conhecidas das notícias.

2. **A2 (2.0)** **Tokenização LLM e Clusterização**. Empregue agora um modelo LLM para tokenização e repita os procedimentos de clusterização anteriores. Apresente brevemente o modelo e suas referências. Analise e compare os resultados indicando a melhor solução encontrada entre o A1 e A2.

3. **A3 (2.0)** **PCA e Clusterização**. Aplique um PCA (0.8) às features (tokens TF-IDF e do LLM) gerando novamente as duas clusterizações selecionadas. Discuta os resultados obtidos.   


## Seleção dos Dados para Clusterização

São selicionados 5 categorias de mensagens empregando o RA do 1o aluno do grupo (ordem alfabética) como `seed` para seleção.

In [ ]:
import numpy as np

RA = 1115665

from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

np.random.seed(RA)
my_categories =  []

for i in np.random.choice(len(categories), 5, replace=False):
    my_categories.append(categories[i])

my_categories

['rec.autos',
 'comp.os.ms-windows.misc',
 'talk.politics.guns',
 'soc.religion.christian',
 'misc.forsale']

In [ ]:
dataset = fetch_20newsgroups(
    remove=("headers", "footers", "quotes"),
    subset="all",
    shuffle=True,
    categories=my_categories,
    random_state=42,
)

In [ ]:
labels = dataset.target
unique_labels, category_sizes = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]

print(f"{len(dataset.data)} documents - {true_k} categories")

In [ ]:
#@markdown
#@title **Avaliação**
A1 = 7 #@param {type:"slider", min:0, max:8, step:1}
#@markdown *Justificativa dos parâmetros TF-IDF e clusterização; aplicação correta dos métodos; análise e discussão dos resultados*

A2 = 2 #@param {type:"slider", min:0, max:2, step:1}
#@markdown *Escolha, descrição e referências do modelo LLM; aplicação correta nos dados; análise e discussão da comparação dos resultados*

A3 = 1.5 #@param {type:"slider", min:0, max:2, step:1}
#@markdown *Escolha e aplicação correta da técnica de PCA; análise e discussão dos resultados*




In [ ]:
#@markdown # **Nota Final**
nota = A1 + A2 + A3

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,7):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['ra'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)

Parte 1

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import adjusted_rand_score, silhouette_score
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score

tfidf = TfidfVectorizer(stop_words='english', max_df=0.5, min_df=5, ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(dataset.data)

def eval_clustering(X, clusters, name=""):
    print(f"{name}:")
    print(f"ARI: {adjusted_rand_score(labels, clusters):.3f}")
    print(f"Silhouette: {silhouette_score(X, clusters):.3f}")
    print(f"Homogeneity: {homogeneity_score(labels, clusters):.3f}")
    print(f"Completeness: {completeness_score(labels, clusters):.3f}")
    print(f"V-Measure: {v_measure_score(labels, clusters):.3f}\n")

kmeans_tfidf = KMeans(n_clusters=true_k, random_state=42, n_init=20)
clusters_kmeans_tfidf = kmeans_tfidf.fit_predict(X_tfidf)
eval_clustering(X_tfidf, clusters_kmeans_tfidf, "TF-IDF + KMeans")

agg_tfidf = AgglomerativeClustering(n_clusters=true_k, linkage='ward')
clusters_agg_tfidf = agg_tfidf.fit_predict(X_tfidf.toarray())
eval_clustering(X_tfidf.toarray(), clusters_agg_tfidf, "TF-IDF + Agglomerative")

Nessa primeira parte nós utilizamos a tokenização TF-IDF (Term Frequency–Inverse Document Frequency) para transformar os textos em vetores numéricos.
Os parâmetros escolhidos foram stop_words='english', max_df=0.5, min_df=5 e ngram_range=(1,2), pois ajudam a deixar a representação mais equilibrada — removendo palavras muito frequentes, ignorando termos raros e considerando combinações de duas palavras (bigramas).

Aplicamos dois algoritmos de clusterização: K-Means e Agglomerative Clustering (linkage = 'ward'). O K-Means foi escolhido pela eficiência e bom desempenho em dados de alta dimensionalidade, enquanto o Agglomerative foi usado para observar possíveis relações hierárquicas entre os textos.

Os resultados mostraram que o K-Means apresentou o melhor desempenho, com ARI = 0.316, Homogeneity = 0.491 e V-Measure = 0.526, enquanto o Agglomerative obteve ARI = 0.239.
O Silhouette Score, próximo de zero (0.008), é comum em textos representados por TF-IDF, já que os vetores são muito esparsos e os clusters acabam ficando próximos uns dos outros.

Em termos de interpretação, o TF-IDF conseguiu separar parcialmente alguns temas (como religião, política e tecnologia), mas ainda ocorreu mistura entre categorias com vocabulário parecido, como rec.autos e misc.forsale.

No geral, o TF-IDF funcionou bem como uma primeira abordagem, conseguindo capturar padrões lexicais, mas sem entender o significado das palavras.
O K-Means se mostrou o modelo mais adequado nessa etapa, representando melhor os grupos em comparação ao Agglomerative.

Parte 2

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans, AgglomerativeClustering

model = SentenceTransformer('all-MiniLM-L6-v2')
X_llm = model.encode(dataset.data, show_progress_bar=True)

kmeans_llm = KMeans(n_clusters=true_k, random_state=42, n_init=50)
clusters_kmeans_llm = kmeans_llm.fit_predict(X_llm)
eval_clustering(X_llm, clusters_kmeans_llm, "LLM + KMeans")

agg_llm = AgglomerativeClustering(
    n_clusters=true_k,
    metric='cosine',
    linkage='average'
)
clusters_agg_llm = agg_llm.fit_predict(X_llm)
eval_clustering(X_llm, clusters_agg_llm, "LLM + Agglomerative")

Nessa segunda parte nós utilizamos um modelo de linguagem (LLM) para gerar representações vetoriais (embeddings) dos textos, substituindo a abordagem de tokenização com TF-IDF.
O modelo usado foi o all-MiniLM-L6-v2, da biblioteca Sentence-Transformers, que gera vetores densos de 384 dimensões e é baseado no Sentence-BERT (Reimers & Gurevych, 2019). Esse modelo consegue capturar o significado e o contexto das frases, e não apenas a frequência das palavras.

Aplicamos novamente os dois algoritmos de clusterização — K-Means e Agglomerative (métrica = 'cosine', linkage = 'average') — para comparar os resultados com a etapa anterior.

O K-Means apresentou um resultado bem superior ao TF-IDF, com ARI = 0.565, Homogeneity = 0.566 e V-Measure = 0.596, mostrando uma separação mais coerente entre os grupos.
Já o Agglomerative teve desempenho um pouco inferior, com ARI = 0.378, Homogeneity = 0.414 e V-Measure = 0.504, mas ainda melhor do que na etapa com TF-IDF.
O Silhouette Score continuou baixo (entre 0.04 e 0.07), o que é esperado em embeddings densos, já que os vetores ficam mais próximos entre si no espaço vetorial.

Esses resultados mostram que os embeddings do LLM representam muito melhor o conteúdo dos textos, agrupando mensagens relacionadas mesmo quando usam vocabulário diferente.
Por exemplo, postagens que falam sobre “fé”, “igreja” e “oração” foram agrupadas no mesmo cluster, mesmo sem repetir as mesmas palavras.

No geral, a combinação LLM + K-Means apresentou o melhor desempenho até o momento, mostrando que usar representações semânticas melhora bastante a qualidade da clusterização.

Parte 3

In [ ]:
from sklearn.decomposition import PCA

X_tfidf_dense = X_tfidf.toarray()
pca_tfidf = PCA(n_components=0.8, svd_solver='full', random_state=42)
X_tfidf_pca = pca_tfidf.fit_transform(X_tfidf_dense)

kmeans_tfidf_pca = KMeans(n_clusters=true_k, random_state=42, n_init=30)
clusters_tfidf_pca = kmeans_tfidf_pca.fit_predict(X_tfidf_pca)
eval_clustering(X_tfidf_pca, clusters_tfidf_pca, "TF-IDF + PCA(0.8) + KMeans")

pca_llm = PCA(n_components=0.8, svd_solver='full', random_state=42)
X_llm_pca = pca_llm.fit_transform(X_llm)

kmeans_llm_pca = KMeans(n_clusters=true_k, random_state=42, n_init=30)
clusters_llm_pca = kmeans_llm_pca.fit_predict(X_llm_pca)
eval_clustering(X_llm_pca, clusters_llm_pca, "LLM + PCA(0.8) + KMeans")

Nessa terceira parte nós aplicamos a técnica de PCA (Principal Component Analysis) sobre os vetores TF-IDF e LLM, mantendo 80% da variância explicada (n_components=0.8).
O objetivo dessa etapa foi reduzir a dimensionalidade dos dados para diminuir o ruído, melhorar o desempenho computacional e verificar o impacto dessa redução na qualidade dos agrupamentos.

Após aplicar o PCA, repetimos o processo de clusterização com o K-Means, utilizando as mesmas configurações das etapas anteriores.

Nos resultados, o TF-IDF + PCA(0.8) apresentou um leve decréscimo nas métricas, com ARI = 0.298, Homogeneity = 0.474 e V-Measure = 0.509, o que mostra uma pequena perda de informação causada pela compressão dos vetores.
Já o LLM + PCA(0.8) manteve praticamente o mesmo desempenho da etapa anterior, com ARI = 0.566, Homogeneity = 0.567 e V-Measure = 0.596, mas com menor custo computacional e resultados mais estáveis.

O Silhouette Score permaneceu baixo (entre 0.008 e 0.095), o que é esperado em dados textuais, especialmente após a redução de dimensão.

De modo geral, o PCA mostrou-se mais útil quando aplicado aos embeddings do LLM, que conseguiram preservar bem a estrutura semântica dos dados mesmo após a redução.
No caso do TF-IDF, a diminuição das dimensões acabou removendo termos discriminativos importantes, o que afetou um pouco a separação entre as categorias.

Assim, concluímos que a combinação LLM + PCA(0.8) + K-Means apresentou o melhor equilíbrio entre desempenho, estabilidade e eficiência computacional, sendo a mais adequada para representar os textos de forma compacta e coerente.